In [ ]:
# 02_optimize_routes.ipynb

from typing import List, Tuple
import pandas as pd
import numpy as np
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp
import math


def haversine_distance(
    coord1: Tuple[float, float], 
    coord2: Tuple[float, float]
) -> float:
    """
    Calculate the Haversine distance between two points in kilometers.

    Args:
        coord1 (Tuple[float, float]): (latitude, longitude) of first point.
        coord2 (Tuple[float, float]): (latitude, longitude) of second point.

    Returns:
        float: Distance in kilometers.
    """
    R = 6371.0  # Earth radius in km

    lat1_rad = math.radians(coord1[0])
    lon1_rad = math.radians(coord1[1])
    lat2_rad = math.radians(coord2[0])
    lon2_rad = math.radians(coord2[1])

    dlat = lat2_rad - lat1_rad
    dlon = lon2_rad - lon1_rad

    a = math.sin(dlat / 2) ** 2 + math.cos(lat1_rad) * math.cos(lat2_rad) * math.sin(dlon / 2) ** 2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))

    distance = R * c
    return distance


def create_distance_matrix(locations: pd.DataFrame) -> List[List[int]]:
    """
    Create a distance matrix between all delivery points as integer meters.

    Args:
        locations (pd.DataFrame): DataFrame with 'latitude' and 'longitude' columns.

    Returns:
        List[List[int]]: 2D list representing the distance matrix in meters.
    """
    coords = locations[['latitude', 'longitude']].values
    size = len(coords)
    matrix: List[List[int]] = []

    for i in range(size):
        row: List[int] = []
        for j in range(size):
            dist_km = haversine_distance(tuple(coords[i]), tuple(coords[j]))
            dist_m = int(dist_km * 1000)  # convert km to meters as int
            row.append(dist_m)
        matrix.append(row)

    return matrix


def optimize_route(distance_matrix: List[List[int]]) -> List[int]:
    """
    Solve the TSP to find the optimal delivery route visiting all points exactly once.

    Args:
        distance_matrix (List[List[int]]): Symmetric matrix of distances between points.

    Returns:
        List[int]: Ordered list of indices representing the visiting order.
    """
    # Create the routing index manager
    manager = pywrapcp.RoutingIndexManager(len(distance_matrix), 1, 0)  # 1 vehicle, depot at index 0

    # Create Routing Model
    routing = pywrapcp.RoutingModel(manager)

    def distance_callback(from_index: int, to_index: int) -> int:
        # Convert routing variable indices to distance matrix indices
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return distance_matrix[from_node][to_node]

    transit_callback_index = routing.RegisterTransitCallback(distance_callback)

    # Define cost of each arc
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

    # Setting first solution heuristic: PATH_CHEAPEST_ARC
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC
    )

    # Solve the problem
    solution = routing.SolveWithParameters(search_parameters)

    if solution:
        route: List[int] = []
        index = routing.Start(0)
        while not routing.IsEnd(index):
            node_index = manager.IndexToNode(index)
            route.append(node_index)
            index = solution.Value(routing.NextVar(index))
        route.append(manager.IndexToNode(index))  # add depot at end
        return route
    else:
        raise RuntimeError("No solution found for the routing problem.")


# Main Execution
if __name__ == "__main__":
    # Load locations
    locations_path: str = "data/locations.csv"
    locations_df: pd.DataFrame = pd.read_csv(locations_path)

    # Create distance matrix
    dist_matrix: List[List[int]] = create_distance_matrix(locations_df)

    # Optimize route
    optimal_route_indices: List[int] = optimize_route(dist_matrix)

    # Map back to locations with IDs for readability
    optimal_route_ids: List[int] = locations_df.iloc[optimal_route_indices]["id"].tolist()

    print("Optimal delivery route order (by location IDs):")
    print(optimal_route_ids)
